# Explainers Notebook

 ## Motivation 

### What is your dataset?

The data we decided to use for our analysis, contains the Star Wars Movie Transcipts for all of the episodes (1-8). And in addition to this we have also downloaded all of the Wookiepedia pages for the charcters who are part of the canonical story. This means the characters from the 8 main movies (Episode 1-8), the two stand alone movies (Rogue One and Solo a Star Wars Story) and the two animated series (The Clones Wars and Rebels). This ends up being a total of 408 Wookiepedia articles. The total size of the data is 12.8 MB.
    

### Why did you choose this/these particular dataset(s)?

We decided on using these two datasets because they hold all the information we need to get out the information we want. The transcript we choose as they hold all the information given in the movies, which is what most casual Star Wars fans only has knowledge of. These are good for analysing sentiments and other stuff.
However, it is very difficult to get any sort of relation between the characters out of the scripts therefore we decided to also download the Wookiepedia articles, as these through the use of hyperlinks will tell us how the characters are connected, according to the fans writing these articles. Also from the Wookiepedia articles we also get some extra data about characters such as race and affiliations, which might not be available through the transcripts.
    
### What was your goal for the end user's experience?

The goal of the project to get a insight to the universe which might not be obvious to the average fan, through analyzing communities, sentiments and nodedegrees. Hopefully we will be able to get some knowledge which will enlighten and educate the user in how the universe is actually working.

## The Dataset
 
### Data cleaning and preprocessing

As our data consist of two sets, we will first explain how we got and preprocessed the movie transcripts, and afterwards for the Wookiepedia articles.

The moivetranscripts come from two sources, IMSDb (https://www.imsdb.com) and the Transcripts wiki (http://transcripts.wikia.com/wiki/Transcripts_Wiki). This is due to neither of the two having all of the scripts, at the time of writing this. The scripts from IMSDB we first downloaded as HTML files, and then removed all of the HTML code to only get the script text. This was done using regular expressions. The scripts we found on the transcripts wiki, has been downloaded as other wiki data through the use of an API. When we had all of the text dataloaded, we had to do some more stuff that Lasse knows more about. (Should we have the preprocessing code here???)

For the Wookiepedia pages we first had to get a list of all the canonical characters. Unfortunately Wookiepedia does not have such a list, and we had to find it on the Wikipedia page for List of Star Wars characters (https://en.wikipedia.org/wiki/List_of_Star_Wars_characters). This however posed a few unexpected challenges. First of all we downloaded the page through the use of the wikipedia API. Most of the characters names was the written in a set of brackets like {{visible anchor | name}}. This made it fairly easy to the the names by using some regular expressions. However not all of the names in the list are the same as the names used on Wookiepedia. This meant we had to removed titles such as Admiral, or only use the first or last name of a character. This however resulted in us getting to many names, as some of the last names of characters, like (Jabba the) Hutt, is the name of a species and therefore has its own Wookiepedia page. These we ended up removing manually. The last hurdle when working with the Wikipedia page, was that two characters names did not follow the earlier mentioned structure, meaning we had to add these manually as well. These two where Anakin Skywalker and Darth Sidious, and as you will learn later are rather important for the network. In the end we ended up with 408 names, which have all been saved in a CSV file for convinence. 

In addition to the names we also add the race of the characters and their "goodness" in the CSV file. The race was found on the Wokkiepedia page, and the good is calculated on a scale based on how many an which affiliations the characters have on their pages. Where an evil affiliation will count as -1 and a good affiliation counts as +1. Affiliations being good or evil is based on whether they are connected to the Sith (evil) or the Jedi(good). (Again should we add the preprocessing code here???)
    


### Dataset stats

So in total we ended up with a CSV file containing 408 rows and four columns (Name, Wookiepedia link, Race and Goodness), 408 Wookiepedia articles and 8 movies transcripts. This total up to 12.8 MB of data. 

 ## Tools, theory and analysis
 
     Talk about how you've worked with text, including regular expressions, unicode, etc.
 
     Describe which network science tools and data analysis strategies you've used, how those network science measures work, and why the tools you've chosen are right for the problem you're solving.
     
    How did you use the tools to understand your dataset?

 
 ### Overall idea
 

 

 

 ### Analysis step 1
 
 #### Explain what you're interested in
     
     Interested in creating a network over the characters, to analyze connections to see who is the most important person, analyze communities to see if these follow any grouping of the actual stuff such as race, movie, alliance, family or other.
 
 #### Explain the tool
 
 #### Apply the tool
 
 #### Discuss the outcome

 ### Analysis step 2
 
 #### Explain what you're interested in
 
 #### Explain the tool
 
 #### Apply the tool
 
 #### Discuss the outcome

 ## Discussion 
 